In [1]:
import talib
import numpy as np
import pandas as pd

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
# df = pd.read_csv(file_path + 'test1.csv')
# df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5.csv')
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_1-1.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2016-07-22 07:00:00   1.32380   1.32424  1.32276    1.32300   1.32407   
1 2016-07-22 07:05:00   1.32300   1.32326  1.32197    1.32279   1.32323   
2 2016-07-22 07:10:00   1.32276   1.32322  1.32230    1.32293   1.32299   
3 2016-07-22 07:15:00   1.32287   1.32322  1.32202    1.32276   1.32312   
4 2016-07-22 07:20:00   1.32282   1.32316  1.32221    1.32272   1.32306   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.32454  1.32299    1.32324   1.32394   1.32438  1.32288    1.32312  
1   1.32350  1.32222    1.32301   1.32312   1.32338  1.32210    1.32290  
2   1.32347  1.32253    1.32316   1.32288   1.32334  1.32242    1.32304  
3   1.32345  1.32224    1.32301   1.32300   1.32334  1.32213    1.32288  
4   1.32339  1.32245    1.32295   1.32294   1.32328  1.32234    1.32284  
---------------------------------------------------------------------------
                     Date  Bi

In [6]:
def add_fractal(df, i, look_back=3):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())

0.0    29721
1.0     3344
2.0     3316
Name: fractal, dtype: int64


In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    for i in range(2, df.shape[0]):
        if i < last_x_bars:
            continue

        ema200 = df.loc[df.index[i - 1], 'ema200']
        atr = df.loc[df.index[i - 1], 'atr']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = (spread / atr) <= atr_percentage
        curr_date = df.loc[df.index[i], 'Date']
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        previous_ask_lows = []
        previous_bid_highs = []
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]

        for ind in range(i - last_x_bars, i):
            previous_ask_lows.append(df.loc[df.index[ind], 'Ask_Low'])
            previous_bid_highs.append(df.loc[df.index[ind], 'Bid_High'])

        if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200 and enough_volatility and trade is None and curr_al <= min(previous_ask_lows) and curr_bar_length <= bar_length:
            pullback = None
            j = i - 4

            while j >= 0:
                if df.loc[df.index[j], 'fractal'] == 1:
                    curr_pullback = float(df.loc[df.index[j], 'Bid_Low'])
                    curr_fractal_distance = curr_ao - curr_pullback

                    if curr_fractal_distance >= fractal_distance:
                        pullback = curr_pullback
                        break

                j -= 1

            if pullback is not None:
                curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                               'Ask_Low', 'Ask_Close']]
                open_price = float(curr_ask_open)
                stop_loss = round(pullback - pullback_cushion, 5)
                # stop_loss = round(pullback - pullback_cushion, 3)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if curr_pips_to_risk <= max_pips_to_risk:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                        # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                 'n_units': n_units, 'start_date': curr_date}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200 and enough_volatility and trade is None and curr_bh >= max(previous_bid_highs) and curr_bar_length <= bar_length:
            pullback = None
            j = i - 4

            while j >= 0:
                if df.loc[df.index[j], 'fractal'] == 2:
                    curr_pullback = float(df.loc[df.index[j], 'Ask_High'])
                    curr_fractal_distance = curr_pullback - curr_bo

                    if curr_fractal_distance >= fractal_distance:
                        pullback = curr_pullback
                        break

                j -= 1

            if pullback is not None:
                curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                               'Ask_Low', 'Ask_Close']]
                open_price = float(curr_bid_open)
                stop_loss = round(pullback + pullback_cushion, 5)
                # stop_loss = round(pullback + pullback_cushion, 3)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if curr_pips_to_risk <= max_pips_to_risk:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                        # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                 'n_units': n_units, 'start_date': curr_date}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)

        if trade is not None:
            curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
            df.loc[
                df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                   'Ask_Close']]

            if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
                trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
                # trade_amount = (trade['stop_loss'] - trade['open_price']) * 100 * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None

                continue

            if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
                trade['stop_loss'] = curr_bid_high - trade['pips_risked']

            if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
                trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
                # trade_amount = (trade['stop_gain'] - trade['open_price']) * 100 * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None

                continue

            if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
                trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
                # trade_amount = (trade['open_price'] - trade['stop_loss']) * 100 * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None

                continue

            if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
                trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

            if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
                trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
                # trade_amount = (trade['open_price'] - trade['stop_gain']) * 100 * value_per_pip
                reward += trade_amount
                day_fees += calculate_day_fees(trade['start_date'], curr_date)

                n_wins += 1 if trade_amount > 0 else 0
                n_losses += 1 if trade_amount < 0 else 0
                curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
                curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

                if curr_win_streak > win_streak:
                    win_streak = curr_win_streak

                if curr_loss_streak > loss_streak:
                    loss_streak = curr_loss_streak

                trade = None

                continue

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [14]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 1.6, 1.7, 1.8, 1.9, 2]
# risk_reward_ratio_vals = [1.6]
pullback_cushion_vals = [10, 15, 20, 25, 30, 35, 40]
# pullback_cushion_vals = [15]
# atr_percentages = [0.25, 0.30, 0.35]
atr_percentages = [0.30]
last_x_bars_vals = [2, 3, 4]
# last_x_bars_vals = [4]
fractal_distances = [0, 5, 10, 15]
bar_lengths = [5, 10, 15, np.inf]
n_runs = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(atr_percentages) * len(last_x_bars_vals) * len(fractal_distances) * len(bar_lengths)
best_risk_reward = None
best_pullback_cushion = None
best_atr_percentage = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
      for atr_percentage in atr_percentages:
          for last_x_bars in last_x_bars_vals:
            for fractal_distance in fractal_distances:
              for bar_length in bar_lengths:
                reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length)
                runs_finished += 1

                print(risk_reward_ratio)
                print(pullback_cushion)
                print(reward)
                print('Num buys: ' + str(n_sells))
                print('Num sells: ' + str(n_buys))
                print('Num trades: ' + str(n_buys + n_sells))
                print('Num wins: ' + str(n_wins))
                print('Num losses: ' + str(n_losses))
                print('Win streak: ' + str(win_streak))
                print('Loss streak: ' + str(loss_streak))
                print('Remaining runs: ' + str(n_runs - runs_finished))

                if reward > best_reward:
                  best_reward = reward
                  best_risk_reward = risk_reward_ratio
                  best_pullback_cushion = pullback_cushion
                  best_atr_percentage = atr_percentage
                  best_last_x_bars_val = last_x_bars
                  best_fractal_distance = fractal_distance
                  best_bar_length = bar_length

                print('Best reward so far: ' + str(best_reward))
                print()

1.5
10
-334.0000000000126
Num buys: 5
Num sells: 2
Num trades: 7
Num wins: 2
Num losses: 5
Win streak: 2
Loss streak: 5
Remaining runs: 2015
Best reward so far: -334.0000000000126

1.5
10
-647.000000000028
Num buys: 12
Num sells: 7
Num trades: 19
Num wins: 6
Num losses: 13
Win streak: 2
Loss streak: 4
Remaining runs: 2014
Best reward so far: -334.0000000000126

1.5
10
-1075.5000000000198
Num buys: 16
Num sells: 9
Num trades: 25
Num wins: 7
Num losses: 18
Win streak: 2
Loss streak: 4
Remaining runs: 2013
Best reward so far: -334.0000000000126

1.5
10
-1538.000000000044
Num buys: 23
Num sells: 12
Num trades: 35
Num wins: 10
Num losses: 25
Win streak: 3
Loss streak: 7
Remaining runs: 2012
Best reward so far: -334.0000000000126

1.5
10
-459.00000000000347
Num buys: 5
Num sells: 2
Num trades: 7
Num wins: 2
Num losses: 5
Win streak: 2
Loss streak: 5
Remaining runs: 2011
Best reward so far: -334.0000000000126

1.5
10
-1026.0000000000155
Num buys: 12
Num sells: 7
Num trades: 19
Num wins: 5
Num

In [15]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best atr percentage: ' + str(best_atr_percentage))
print('Best last x bars: ' + str(best_last_x_bars_val))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))

------------ FINAL RESULTS ------------
Best reward: 627.9999999999915
Best risk/reward ratio: 2
Best pullback cushion: 35
Best atr percentage: 0.3
Best last x bars: 3
Best fractal distance: 5
Best bar length: 10
